In [1]:
!pip -q install langchain huggingface_hub transformers sentence_transformers accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 104.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.4 MB/s eta 0

In [2]:
from langchain.llms import HuggingFacePipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM

model_id = 'google/flan-t5-large'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=100
)

local_llm = HuggingFacePipeline(pipeline=pipe)

In [3]:
from langchain.embeddings import HuggingFaceEmbeddings
model_name = "sentence-transformers/all-mpnet-base-v2"
hf = HuggingFaceEmbeddings(model_name=model_name)

In [4]:
pip install requests

In [6]:
import requests

def download_pdf_from_url(url, save_path):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(save_path, 'wb') as pdf_file:
            for chunk in response.iter_content(1024):
                pdf_file.write(chunk)
        print(f"PDF downloaded successfully and saved at: {save_path}")
    else:
        print(f"Failed to download PDF. Status code: {response.status_code}")

pdf_url = 'https://www.iitb.ac.in/newacadhome/ugrulebook.pdf'
save_path = 'downloaded_file.pdf'

download_pdf_from_url(pdf_url, save_path)


PDF downloaded successfully and saved at: downloaded_file.pdf


In [7]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.6/276.6 kB 4.2 MB/s eta 0:00:00


In [9]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("downloaded_file.pdf")
pages=loader.load()

In [10]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=150,
    length_function=len
)
docs = text_splitter.split_documents(pages)

In [11]:
pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.1 MB/s eta 0:00:00


In [12]:
from langchain.vectorstores import FAISS
vectorStore = FAISS.from_documents(docs, hf)

In [13]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    local_llm,
    retriever=vectorStore.as_retriever()
)

In [17]:
prompt='What is Minor.'
result=qa_chain({'query':prompt})
print(result['result'])

Minor is an additional credential a student may earn if s/he does 30 credits worth of additional courses in a discipline other than her/his major discipline of B.Tech. degree.


In [18]:
prompt='What is Honours.'
result=qa_chain({'query':prompt})
print(result['result'])

an additional credential a student may earn if s/he opts for the extra 24 credits needed for this in her/his own discipline


In [19]:
prompt='What are some roles of the Faculty Adviser?'
result=qa_chain({'query':prompt})
print(result['result'])

i. Guide the students about the rules and regulations governing the courses of study. ii. Advise the students to register for courses as per the curriculum and their academic standing. iii. Approve the registration of the students. iv. Pay special attention to weak students and carefully monitor the performance of students rec- ommended for the slow track option. v. Advice students on Course Adjustment / Drop during the Semester within


In [21]:
prompt='What happens when a student is registered late.'
result=qa_chain({'query':prompt})
print(result['result'])

Late Registration Only for valid reasons, late registration may be permitted till the date specified in the Academic Calendar, on payment of a late registration fee


In [22]:
prompt='Tell me in short about the registeration of NPTEL courses.'
result=qa_chain({'query':prompt})
print(result['result'])

A student can take maximum of 12 credits of NPTEL/SWAYAM courses in lieu of departmental elective and Institute elective courses for the regular student (not under ARP), and, (b) a maximum of 24 credits of NPTEL/SWAYAM courses in lieu of any courses including core courses for the students, who are under ARP or in extension years after the approval from DUGC
